In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [150]:
df=pd.read_csv("Dataset .csv")

In [151]:
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [152]:
df['City'].unique()

array(['Makati City', 'Mandaluyong City', 'Pasay City', 'Pasig City',
       'Quezon City', 'San Juan City', 'Santa Rosa', 'Tagaytay City',
       'Taguig City', 'Bras�_lia', 'Rio de Janeiro', 'S��o Paulo',
       'Albany', 'Armidale', 'Athens', 'Augusta', 'Balingup',
       'Beechworth', 'Boise', 'Cedar Rapids/Iowa City', 'Chatham-Kent',
       'Clatskanie', 'Cochrane', 'Columbus', 'Consort', 'Dalton',
       'Davenport', 'Des Moines', 'Dicky Beach', 'Dubuque',
       'East Ballina', 'Fernley', 'Flaxton', 'Forrest', 'Gainesville',
       'Hepburn Springs', 'Huskisson', 'Inverloch', 'Lakes Entrance',
       'Lakeview', 'Lincoln', 'Lorn', 'Macedon', 'Macon', 'Mayfield',
       'Mc Millan', 'Middleton Beach', 'Miller', 'Monroe', 'Montville',
       'Ojo Caliente', 'Orlando', 'Palm Cove', 'Paynesville', 'Penola',
       'Pensacola', 'Phillip Island', 'Pocatello', 'Potrero', 'Princeton',
       'Rest of Hawaii', 'Savannah', 'Singapore', 'Sioux City',
       'Tampa Bay', 'Tanunda', 'Trentha

In [155]:
df['Cuisines'] = df['Cuisines'].fillna("unknown")
df['Price range'] = df['Price range'].fillna(df['Price range'].median())


In [156]:
df['Combined'] = df['Cuisines'].str.lower() + " " + df['Price range'].astype(str)


In [157]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
feature_matrix = vectorizer.fit_transform(df['Combined'])


In [169]:
user_cuisine = "Chinese"
user_price_range = 3
user_input = f"{user_cuisine.lower()} {user_price_range}"


In [170]:
from sklearn.metrics.pairwise import cosine_similarity

user_vector = vectorizer.transform([user_input])
df['Similarity'] = cosine_similarity(user_vector, feature_matrix)[0]


In [171]:
# recommendations = df.sort_values(by='Similarity', ascending=False).head(5)
recommendations = (
    df.sort_values(by=['Similarity', 'Aggregate rating'], ascending=[False, False])
    .head(5)
)

print(recommendations[['Restaurant Name', 'City', 'Cuisines', 'Price range', 'Aggregate rating', 'Similarity']])


                       Restaurant Name              City Cuisines  \
9424         Mainland China Restaurant              Doha  Chinese   
5                         Din Tai Fung  Mandaluyong City  Chinese   
785                     Mainland China      Bhubaneshwar  Chinese   
2296                          Chung Fa          Guwahati  Chinese   
9308  Eden Noodles Cafe �__·�_��_��_��          Auckland  Chinese   

      Price range  Aggregate rating  Similarity  
9424            4               4.9         1.0  
5               3               4.4         1.0  
785             3               4.3         1.0  
2296            2               4.3         1.0  
9308            2               4.3         1.0  


In [174]:
def test_recommendation_system(user_cuisine, user_price_range):
    user_input = f"{user_cuisine.lower()} {user_price_range}"
    user_vector = vectorizer.transform([user_input])
    df['Similarity'] = cosine_similarity(user_vector, feature_matrix)[0]
    
    recommendations = (
        df.sort_values(by=['Similarity', 'Aggregate rating'], ascending=[False, False])
        .head(5)
    )
    return recommendations[['Restaurant Name', 'City', 'Cuisines', 'Price range', 'Aggregate rating', 'Similarity']]
    

In [175]:
test_cases = [
    {"cuisine": "Japanese", "price_range": 4},
    {"cuisine": "Italian", "price_range": 3},
    {"cuisine": "Seafood", "price_range": 4}
]

# Generate Recommendations for Each User
for idx, case in enumerate(test_cases, 1):
    print(f"--- User {idx} ---")
    print(f"Preferences: {case}")
    results = test_recommendation_system(case['cuisine'], case['price_range'])
    print(results, "\n")

--- User 1 ---
Preferences: {'cuisine': 'Japanese', 'price_range': 4}
       Restaurant Name            City  Cuisines  Price range  \
429      Marukame Udon  Rest of Hawaii  Japanese            1   
54        Sushi Leblon  Rio de Janeiro  Japanese            4   
1     Izakaya Kikufuji     Makati City  Japanese            3   
222       Soho Hibachi          Dalton  Japanese            1   
9289    3 Wise Monkeys         Jakarta  Japanese            3   

      Aggregate rating  Similarity  
429                4.9         1.0  
54                 4.6         1.0  
1                  4.5         1.0  
222                4.3         1.0  
9289               4.2         1.0   

--- User 2 ---
Preferences: {'cuisine': 'Italian', 'price_range': 3}
      Restaurant Name             City Cuisines  Price range  \
811   Fusilli Reasons          Chennai  Italian            1   
9316          Baduzzi         Auckland  Italian            4   
943     Cafe Parmesan        Faridabad  Italian       